In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('/Users/aoamacsplace/Documents/mosaic/Aave/token/data_copy.csv')
df.columns

Index(['Unnamed: 0', 'id', 'block_number', 'time_stamp', 'hash', 'nonce',
       'block_hash', 'transaction_index', 'From', 'to', 'value', 'gas',
       'gas_price', 'is_error', 'txreceipt_status', 'input',
       'contract_address', 'cumulative_gas_used', 'gas_used', 'confirmations',
       'method_id', 'function_name', 'token_name', 'token_symbol',
       'token_decimal', 'token_address', 'Method'],
      dtype='object')

In [ ]:



deposit_value_intervals = [0, 1, 10, 100, 1000, 10000]
deposit_value_scores = [10, 25, 40, 55, 70, 100]

deposit_time_intervals = [0, 30, 90, 180, 365]
deposit_time_scores = [10, 30, 50, 70, 100]

borrow_value_intervals = [0, 1, 10, 100, 1000, 10000]
borrow_value_scores = [10, 25, 40, 55, 70, 100]

borrow_time_intervals = [0, 30, 90, 180, 365]
borrow_time_scores = [10, 30, 50, 70, 100]

repayment_value_intervals = [0, 1, 10, 100, 1000, 10000]
repayment_value_scores = [10, 25, 40, 55, 70, 100]

deposit_df = pd.DataFrame(columns=['Address', 'value', 'Deposit Value Score'])
deposit_time_df = pd.DataFrame(columns=['Address', 'DateTime', 'Deposit Time Score'])
borrow_df = pd.DataFrame(columns=['Address', 'value', 'Borrow Value Score'])
borrow_time_df = pd.DataFrame(columns=['Address', 'DateTime', 'Borrow Time Score'])
repayment_df = pd.DataFrame(columns=['Address', 'value', 'Repayment Value Score'])

for address, group_data in df.groupby('From'):
    methods = group_data['Method'].unique()
    for method in methods:
        if 'depositETH' in method:
            value_in_eth = group_data['value'].sum()
            deposit_value_score = None
            deposit_time_score = None

           
            if 0 <= value_in_eth < 1:
                deposit_value_score = 10
            elif 1 <= value_in_eth < 10:
                deposit_value_score = 25
            elif 10 <= value_in_eth < 100:
                deposit_value_score = 40
            elif 100 <= value_in_eth < 1000:
                deposit_value_score = 55
            elif 1000 <= value_in_eth < 10000:
                deposit_value_score = 70
            else:
                deposit_value_score = 100

            deposit_df = pd.concat([deposit_df, pd.DataFrame({'Address': [address], 'value': [value_in_eth], 'Deposit Value Score': [deposit_value_score]})], ignore_index=True)

            days_since_deposit = (pd.Timestamp.now() - pd.to_datetime(group_data['DateTime'])).dt.days
            if days_since_deposit.iloc[0] <= 30:
                deposit_time_score = 10
            elif days_since_deposit.iloc[0] <= 90:
                deposit_time_score = 30
            elif days_since_deposit.iloc[0] <= 180:
                deposit_time_score = 50
            elif days_since_deposit.iloc[0] <= 365:
                deposit_time_score = 70
            else:
                deposit_time_score = 100

            deposit_time_df = pd.concat([deposit_time_df, pd.DataFrame({'Address': [address], 'DateTime': [group_data['DateTime'].iloc[0]], 'Deposit Time Score': [deposit_time_score]})], ignore_index=True)

        elif 'borrowETH' in method:
            value_out_eth = group_data['value'].sum()
            borrow_value_score = None
            borrow_time_score = None

         
            if 0 <= value_out_eth < 1:
                borrow_value_score = 10
            elif 1 <= value_out_eth < 10:
                borrow_value_score = 25
            elif 10 <= value_out_eth < 100:
                borrow_value_score = 40
            elif 100 <= value_out_eth < 1000:
                borrow_value_score = 55
            elif 1000 <= value_out_eth < 10000:
                borrow_value_score = 70
            else:
                borrow_value_score = 100

            borrow_df = pd.concat([borrow_df, pd.DataFrame({'Address': [address], 'value': [value_out_eth], 'Borrow Value Score': [borrow_value_score]})], ignore_index=True)

        
            days_since_borrow = (pd.Timestamp.now() - pd.to_datetime(group_data['DateTime'])).dt.days
            if days_since_borrow.iloc[0] <= 30:
                borrow_time_score = 10
            elif days_since_borrow.iloc[0] <= 90:
                borrow_time_score = 30
            elif days_since_borrow.iloc[0] <= 180:
                borrow_time_score = 50
            elif days_since_borrow.iloc[0] <= 365:
                borrow_time_score = 70
            else:
                borrow_time_score = 100
 
            borrow_time_df = pd.concat([borrow_time_df, pd.DataFrame({'Address': [address], 'DateTime': [group_data['DateTime'].iloc[0]], 'Borrow Time Score': [borrow_time_score]})], ignore_index=True)

        elif 'Repay ETH' in method:
            value_in_eth = group_data['value'].sum()
            repayment_value_score = None

         
            if 0 <= value_in_eth < 1:
                repayment_value_score = 10
            elif 1 <= value_in_eth < 10:
                repayment_value_score = 25
            elif 10 <= value_in_eth < 100:
                repayment_value_score = 40
            elif 100 <= value_in_eth < 1000:
                repayment_value_score = 55
            elif 1000 <= value_in_eth < 10000:
                repayment_value_score = 70
            else:
                repayment_value_score = 100

            repayment_df = pd.concat([repayment_df, pd.DataFrame({'Address': [address], 'value': [value_in_eth], 'Repayment Value Score': [repayment_value_score]})], ignore_index=True)


lp_scores_df = pd.merge(deposit_df, deposit_time_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, borrow_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, borrow_time_df, on='Address', how='outer')
lp_scores_df = pd.merge(lp_scores_df, repayment_df, on='Address', how='outer')


lp_scores_df['LP Score'] = lp_scores_df[['Deposit Value Score', 'Deposit Time Score', 'Borrow Value Score', 'Borrow Time Score', 'Repayment Value Score']].sum(axis=1)


lp_scores_df['LP Reward'] = (lp_scores_df['LP Score'] / 500) * 100


print("Merged Dataframe:")
print(lp_scores_df)
